In [ ]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import itertools
import matplotlib
import time
import seaborn as sns
from time import sleep
from random import randint
sns.set()

""" This program scrapes KTLA for articles in a specific city entered"""

In [ ]:
#disguise as a browser so website does not block us/identify us as web scraper
headers = ({'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

In [ ]:
titles = []
#tags = []  #still need to find
times = []
authors = []
#descs = []   #does not have description
links = []

In [ ]:
#link to ktla's website
city = "norwalk"  #<--- Enter a city name here
city = city.replace(" ","-")

In [ ]:
n_pages = 0

for page in range(0,5):
    n_pages +=1
    ktla = "https://ktla.com/tag/" + city + "/"
    response = get(ktla, headers = headers)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    article_containers = html_soup.find_all('h2', class_="article-list__article-title")
    article_containers += html_soup.find_all('h3', class_="article-list__article-title")
    
    #need to loop through article containers
    if article_containers != []:
         for container in article_containers:
    
             #getting the title
            title = container.find_all('a')[0].text
            title = title.replace("\t","")
            title = title.replace("\n","")
            titles.append(title)
            print(title)

            #getting the link
            link = container.find_all('a')[0].attrs["href"]
            links.append(link)

            #going to go into article link to get author, date
            ktla_article = link
            article_response = get(ktla_article, headers = headers)
            html_soup_2 = BeautifulSoup(article_response.text, 'html.parser')
            article_info = html_soup_2.find_all('div',class_="article-meta")
            article =article_info[0]

            #getting the author
            author = article.find_all('a')[0].text
            authors.append(author)

             #getting the timestamp
            timestamp = pd.to_datetime(article.time.attrs['datetime'])
            timestamp = timestamp.replace(tzinfo=None)
            times.append(timestamp)


    else:
        break

    sleep(randint(1,2))

print('You scraped {} pages containing {} properties.'.format(n_pages, len(titles)))



cols = ['Title','Date Posted', 'Author','Link']

ktla_pages = pd.DataFrame({'Title':titles, 'Date Posted':times, 'Author':authors, 'Link':links})[cols]

ktla_pages.to_json('ktla.json')
    